# Problem:
## Given a morphology in [Arbor](https://arbor.readthedocs.io/en/latest/index.html#) how does one setup the channels?

### 1. First the arbor-morphology is created
See `PyNN_neuroml_morphology_IN_Arbor.ipynb` for description of the steps.

In [1]:
from neuroml import Morphology, Segment, Point3DWithDiam as P
from pyNN.morphology import NeuroMLMorphology

In [2]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)
neuroml_morph = Morphology(segments=(soma, dend))
pynn_morph = NeuroMLMorphology(neuroml_morph)

In [3]:
import arbor
import re

In [4]:
# methods already defined in PyNN_neuroml_morphology_IN_Arbor.ipynb
def create_arbor_tree(pynn_nml_morph):
    tree = arbor.segment_tree()
    for i, nml_seg in enumerate(pynn_nml_morph.segments):
        append_arbor_tree(tree, nml_seg)
    return tree
    
def append_arbor_tree(tree, nml_seg):
    if not nml_seg.parent:
        tree.append(arbor.mnpos,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter/2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))
    else:
        tree.append(nml_seg.parent.id,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter/2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))

def get_swc_tag(nml_seg):
    if re.search("soma", nml_seg.name, re.IGNORECASE):
        return 1
    elif re.search("axon", nml_seg.name, re.IGNORECASE):
        return 2
    elif re.search("dend", nml_seg.name, re.IGNORECASE):
        return 3
    else:
        return 5

In [5]:
tree = create_arbor_tree(pynn_morph)

In [6]:
#Arbor uses a domains specific language (DSL) to describe regions and locations, which are given labels.
def create_region_definitions(pynn_morph):
    dict_defs = {}
    for i, nml_seg in enumerate(pynn_morph.segments):
        dict_defs.update({nml_seg.name: "(tag "+ str(get_swc_tag(nml_seg))+ ")"})
    dict_defs.update({"everywhere": "(all)"})
    return dict_defs

&#127941; Refer at bottom for `dict_defs.update({"everywhere": "(all)"})`

In [7]:
#dict_defs => {'soma': '(tag 1)', 'dendrite': '(tag 3)'}
tree_labels = arbor.label_dict(create_region_definitions(pynn_morph))

### 2. Create cell

In [8]:
# Cell builders need to refer to regions and locations on a cell morphology.
cell = arbor.cable_cell(tree, tree_labels)

### 3. Set ion properties.
##### 3.1. Set reversal potentials for respective ions.
Although, one might think of this as part of the `arbor.mechanism` it is not. But, one may set the reversal potential for respective ion in the cell created above as follows,

In [9]:
cell.set_ion(ion="na", rev_pot=50.)
cell.set_ion(ion="k", rev_pot=-77.)

Note: In Arbor, the reversal potential can also be [computed based on Nernst equation](https://arbor.readthedocs.io/en/pydoc/cable_cell.html?#ion-species)

### 4. Describe the desired `mechanism`.
`arbor.mechanism` describes physical processes, distributed over the membrane of the cell. There are broadly [three classes of mechanisms](https://arbor.readthedocs.io/en/pydoc/mechanisms.html?#mechanism)
- Density mechanisms are associated with regions of the cell, whose dynamics are a function of the cell state and their own state where they are present.
- Point mechanisms are defined at discrete locations on the cell, which receive events from the network.
- A third, specific type of density mechanism, which describes ionic reversal potential behaviour, can be specified for cells or the whole model.

#### 4.1. Set channel mechanisms.
##### 4.1.1. For leaky channel, i.e. `passive`
For `passive` channel mechanism an instance of `arbor.mechanism('passive')` is created. But, this can be done in various ways:
* Create instance using default parameter values (set in NMODL file)
    - `pas = arbor.mechanism('passive')`
* Create instance with custom reversal potential (Arbor term: [global](https://arbor.readthedocs.io/en/pydoc/mechanisms.html?#mechanism-catalogues))
    - `pas = arbor.mechanism('passive/el=-54.3')`
    - NOTE: in `arbor.mechanism` **el** stands for reversal potential
* Create instance with custom maximum conductance (Arbor term: [range](https://arbor.readthedocs.io/en/pydoc/mechanisms.html?#mechanism-catalogues))
    - `pas = arbor.mechanism('passive', {'g': 0.0003})`
    - NOTE: in `argbor.mechanism` **g** stands for gbar for respective HH-based channel, maximum conductance.
* Create instance with custom reversal potential and custom maximum conductance
    - `pas = arbor.mechanism('passive/el=-54.3', {'g': 0.0003})`
    - Alternatively, this can be done in two steps as
        - `pas = arbor.mechanism('passive/el=-54.3')`
        - `pas.set('g', 0.0003)`

For this example we would like to enforce `pas={"conductance_density": uniform('all', 0.0003), "e_rev":-54.3}`

In [10]:
leaky_chnl = arbor.mechanism('passive/el=-54.3', {'g': 0.0003})

##### 4.1.2. For HH-based Na channel, i.e. `na` and HH-based K channel, i.e. `k`

In [11]:
na_chnl = arbor.mechanism("hh", {"ena": 50., "gnabar": 0.120})
# na={"conductance_density": uniform('soma', 0.120), "e_rev": 50.0}
na_chnl = arbor.mechanism("hh", {"ek": -77., "gkbar": 0.036})
# kdr={"conductance_density": uniform('soma', 0.036), "e_rev": -77.0}
hh_na_k_chnl = arbor.mechanism("hh", {"ena": 50., "gnabar": 0.120, "ek": -77., "gkbar": 0.036, "gl": 0.})

Note that in `hh_na_k_chnl` the value for `"gl"` is set to 0 because our plan is to paint this channel to specific regions of the cell while pain `leaky_chnl` to every region. And, remember that `leaky_chnl` is also a Hodgkin-Huxley based leaky channel therefore painting it twice will result in duplicated passive channels.

&#127941; Refer at bottom for {`"gl"`: 0.} in `hh_na_k_chnl`.

### 5. Attaching the `mechanism` to the cell.
In Arbor attaching to a cell is referred to as [decoration](https://arbor.readthedocs.io/en/pydoc/cable_cell.html?#decoration). On can decorate a cell by _painting_ or _placing_.

| Painting                    | Placing                       |
|:----------------------------|:------------------------------|
| to regions of a cell to set | to locations on a cell to set |
| * Cable properties          | * Synapses                    |
| * Density mechanisms        | * Gap junction sites          |
| * Ion species               | * Threshold (spike) detectors |
|                             | * Stimuli                     |
|                             | * Probes                      |

For our purpose of attaching `arbor.mechanism` one must therefore paint.

In [12]:
cell.paint("soma", hh_na_k_chnl)

Because of the resolution options of _painting_.

| to paint               | region | cell | global |
|:-----------------------|:------:|:----:|:------:|
| cable properties       | ✓      |  ✓  |  ✓     |
| ion initial conditions | ✓      |  ✓  |  ✓     |
| density mechnism       | ✓      | -   | -       |
| ion rev pot mechanism  | -      |  ✓  | ✓      |
| ion valence            | -      | -   |  ✓     |

if the objective is to attach the `passive` mechanism to the whole cell one must loop through each segment.

In [13]:
cell.paint('everywhere', leaky_chnl)

#for i, seg in enumerate(pynn_morph.segments):
#    #print(seg.name)
#    cell.paint(seg.name, leaky_chnl)

&#127941; Note that from Sam Yates (22 Sept 2020)
> Regarding the new release (0.4): we're aiming to have that published within the next two weeks, and some of the changes to the API will hit our master branch before then, which will probably mean that some changes to your python code will be required too I'm afraid.

```
cell_type = cell_class(morphology=NeuroMLMorphology(Morphology(segments=(soma, dend))),  # yuck
                       cm=1.0,
                       Ra=500.0,
                       ionic_species={
                              "na": IonicSpecies("na", reversal_potential=50.0),
                              "k": IonicSpecies("k", reversal_potential=-77.0)
                       },
                       pas={"conductance_density": uniform('all', 0.0003),
                            "e_rev":-54.3},
                       na={"conductance_density": uniform('soma', 0.120),
                           "e_rev": 50.0},
                       kdr={"conductance_density": uniform('soma', 0.036),
                            "e_rev": -77.0}
                       )
```